In [1]:
import torch
import torch.nn as nn
import csv
import json
import math
from torch.autograd import Variable
from pytorch_pretrained_bert.tokenization import BertTokenizer
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from transformer_nb import *

In [2]:
import sys
import json
doc_name = "../pointer-generator/data/Giga/input.txt"
summ_name = "../pointer-generator/data/Giga/task1_ref0.txt"
task_name = "giga"
val_size = 13368

capitalize = False

vocab_name = "data/vocab.json"
continue_from = "trained/Model10" # if none, put None
eval_dir = "evaluation-giga/"
data_seq_name = eval_dir+'tmp.json'


num_threads = 4
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)

In [3]:
INPUT_MAX = 60
OUTPUT_MAX = 20
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda


In [5]:
def getlines(name):
    total = !wc -l {name}
    return int(total[0].split()[0])

In [6]:
summaries = []
articles = []

with open(doc_name, newline='', encoding='utf-8') as f:
    total = getlines(doc_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        articles.append(line)
with open(summ_name, newline='', encoding='utf-8') as f:
    total = getlines(summ_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        summaries.append(line)
        
print(len(summaries))


1951


In [7]:
summaries = summaries[:val_size]
articles = articles[:val_size]
print(len(summaries))

1951


In [8]:
def get_dig(n):
    return 1 if n < 10 else (get_dig(n/10) + 1)

digits = get_dig(val_size)
print(digits)

def make_gold(summ):
    for i,s in tqdm(enumerate(summ), total=len(summ)):
        filename = (eval_dir+'gold/'+task_name+'.A.{:0'+str(digits)+'d}.txt').format(i+1)
        with open(filename, 'w') as f:
            f.write(s)
# make_gold(summaries)

5


In [9]:
if task_name == 'giga':
    token_mappings = {'UNK':UNK, '-lrb-':'(', '-rrb-':')'}
else:
    token_mappings = {'<unk>':UNK, '<t>':'', '</t>':''}
token_mappings_inv = {a:b for b, a in token_mappings.items()}

In [11]:
def task(s):
    for t, t_p in token_mappings.items():
        s = s.replace(t, t_p)
    ts = tokenizer.tokenize(s)
    return [BOS] + ts + [EOS]

In [12]:
stmp = "india 's congress -lrb- i -rrb- party thursday said it \
would block some of the government 's UNK tax cut plans ."
print(task(stmp))

['[CLS]', 'india', "'", 's', 'congress', '(', 'i', ')', 'party', 'thursday', 'said', 'it', 'would', 'block', 'some', 'of', 'the', 'government', "'", 's', '[UNK]', 'tax', 'cut', 'plans', '.', '[SEP]']


In [13]:
with Pool(num_threads) as p:
    chunksize = 200
    summaries = list(tqdm(p.imap(task, summaries, chunksize=chunksize), total=len(summaries)))
    articles = list(tqdm(p.imap(task, articles, chunksize=chunksize), total=len(articles)))

In [14]:
num_summ = len(summaries)
index = 73
print("[summary]", summaries[index])
print("[documen]", articles[index])

[summary] ['[CLS]', 'russia', 'warns', 'of', 'colossal', 'impact', 'if', 'nato', 'takes', 'in', 'ukraine', 'georgia', '[UNK]', '[UNK]', 'quote', '[SEP]']
[documen] ['[CLS]', 'russia', 'warned', 'wednesday', 'against', 'nato', 'taking', 'in', 'the', 'ex', '-', 'soviet', 'republics', 'of', 'ukraine', 'and', 'georgia', ',', 'saying', 'such', 'a', 'colossal', 'geo', '##pol', '##itical', 'shift', 'would', 'threaten', 'relations', '.', '[SEP]']


In [15]:
seqdata = {'summary':[], 'document':[]}

with Pool(num_threads) as p:
    chunksize = 200
    summ_seqs = list(tqdm(p.imap(tokenizer.convert_tokens_to_ids, summaries, chunksize=chunksize), total=num_summ))
    docu_seqs = list(tqdm(p.imap(tokenizer.convert_tokens_to_ids, articles, chunksize=chunksize), total=num_summ))

for i in range(num_summ):    
    seqdata['summary'].append(summ_seqs[i])
    seqdata['document'].append(docu_seqs[i])

json.dump(seqdata, open(data_seq_name, 'w'))

In [16]:
from dataset import make_data_generator
batch_size = 128
ev_set, ev_generator = make_data_generator(\
data_seq_name, INPUT_MAX, OUTPUT_MAX, tokenizer.vocab[PAD], batch_size, cutoff=None, shuffle=False, num_workers=4)

loading json
load json done.


In [17]:
def data_gen_val():
    for src,tgt in ev_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])

In [18]:
if str(device) == 'cpu':
    saved_model = torch.load(continue_from, map_location=lambda storage, location: storage)
else:
    saved_model = torch.load(continue_from)

model = make_model(VOC_SIZE, VOC_SIZE, N=4, d_model=256, d_ff=1024, h=8, dropout=0.1)
model.load_state_dict(saved_model['model'])
model.eval()
model.cuda()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
            (3): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1024, bias=True)
          (w_2): Linear(in_features=1024, out_features=256, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [19]:
def greedy_decode_batch(model, src, src_mask, max_len, start_symbol):
    batch_size = src.shape[0]
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(batch_size, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        #print(out.shape) 128,1,256
        probs = model.generator(out[:, -1, :])
        
        #print(probs.shape) 128,30522
        next_words = torch.argmax(probs, dim=1, keepdim=True)
        
        #print(next_words.shape)        
        #print(ys.shape) both 128,1
        
        ys = torch.cat((ys, next_words), dim=1)
    return ys

In [20]:
def readable(sent):
    try:
        end = sent.index(EOS)
    except ValueError:
        end = len(sent)
    sent = [tok for tok in sent[:end] if tok not in [BOS, EOS, PAD]] # remove special tokens
    sent = " ".join(sent)
    for t, t_p in token_mappings.items():
        sent = sent.replace(t_p, t)
    return sent

In [21]:
examples = min(100000, ev_set.size)
print(examples)

total = int(math.ceil(examples / batch_size))


for i,batch in tqdm(enumerate(data_gen_val()), total= total):
    srcs = batch.src
    src_masks = batch.src_mask
    
    trgs = batch.trg
    trg_masks = batch.trg_mask
        
    bs = srcs.shape[0]
    
    outs = greedy_decode_batch(model, srcs, None, max_len=20, start_symbol=vocab[BOS])
    
    for j, (out_tensor, tgt_tensor) in enumerate(zip(outs, trgs)):        
        tokens = tokenizer.convert_ids_to_tokens(out_tensor.cpu().numpy())        
        line = readable(tokens)
        #print(line)
        numeric = i*batch_size + j + 1
        filename = (eval_dir+'mine/'+task_name+'.{:0'+str(digits)+'d}.txt').format(numeric)
        
        with open(filename, 'w') as mine:
            mine.write(line)
            
    if i == total:
        break

1951


In [23]:

from pyrouge import Rouge155

r = Rouge155()
r.system_dir = eval_dir+'mine/'
r.model_dir = eval_dir+'gold/'
r.system_filename_pattern = task_name+'.(\d+).txt'
r.model_filename_pattern = task_name+'.[A-Z].#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-06-09 12:07:46,543 [MainThread  ] [INFO ]  Writing summaries.
2019-06-09 12:07:46,545 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp64yn_xfz/system and model files to /tmp/tmp64yn_xfz/model.
2019-06-09 12:07:46,545 [MainThread  ] [INFO ]  Processing files in evaluation-giga/mine/.
2019-06-09 12:07:46,547 [MainThread  ] [INFO ]  Processing giga.01502.txt.
2019-06-09 12:07:46,548 [MainThread  ] [INFO ]  Processing giga.01455.txt.
2019-06-09 12:07:46,549 [MainThread  ] [INFO ]  Processing giga.00103.txt.
2019-06-09 12:07:46,550 [MainThread  ] [INFO ]  Processing giga.00967.txt.
2019-06-09 12:07:46,551 [MainThread  ] [INFO ]  Processing giga.00876.txt.
2019-06-09 12:07:46,552 [MainThread  ] [INFO ]  Processing giga.00458.txt.
2019-06-09 12:07:46,552 [MainThread  ] [INFO ]  Processing giga.01357.txt.
2019-06-09 12:07:46,553 [MainThread  ] [INFO ]  Processing giga.01496.txt.
2019-06-09 12:07:46,554 [MainThread  ] [INFO ]  Processing giga.00178.txt.
2019-06

2019-06-09 12:07:46,629 [MainThread  ] [INFO ]  Processing giga.01784.txt.
2019-06-09 12:07:46,630 [MainThread  ] [INFO ]  Processing giga.00530.txt.
2019-06-09 12:07:46,631 [MainThread  ] [INFO ]  Processing giga.00035.txt.
2019-06-09 12:07:46,632 [MainThread  ] [INFO ]  Processing giga.00948.txt.
2019-06-09 12:07:46,633 [MainThread  ] [INFO ]  Processing giga.00970.txt.
2019-06-09 12:07:46,633 [MainThread  ] [INFO ]  Processing giga.01873.txt.
2019-06-09 12:07:46,634 [MainThread  ] [INFO ]  Processing giga.01365.txt.
2019-06-09 12:07:46,635 [MainThread  ] [INFO ]  Processing giga.00190.txt.
2019-06-09 12:07:46,635 [MainThread  ] [INFO ]  Processing giga.00014.txt.
2019-06-09 12:07:46,636 [MainThread  ] [INFO ]  Processing giga.00104.txt.
2019-06-09 12:07:46,636 [MainThread  ] [INFO ]  Processing giga.00519.txt.
2019-06-09 12:07:46,637 [MainThread  ] [INFO ]  Processing giga.00895.txt.
2019-06-09 12:07:46,637 [MainThread  ] [INFO ]  Processing giga.01429.txt.
2019-06-09 12:07:46,638 [

2019-06-09 12:07:46,716 [MainThread  ] [INFO ]  Processing giga.00370.txt.
2019-06-09 12:07:46,717 [MainThread  ] [INFO ]  Processing giga.00099.txt.
2019-06-09 12:07:46,717 [MainThread  ] [INFO ]  Processing giga.01427.txt.
2019-06-09 12:07:46,718 [MainThread  ] [INFO ]  Processing giga.00506.txt.
2019-06-09 12:07:46,718 [MainThread  ] [INFO ]  Processing giga.00396.txt.
2019-06-09 12:07:46,719 [MainThread  ] [INFO ]  Processing giga.01439.txt.
2019-06-09 12:07:46,720 [MainThread  ] [INFO ]  Processing giga.01246.txt.
2019-06-09 12:07:46,720 [MainThread  ] [INFO ]  Processing giga.00535.txt.
2019-06-09 12:07:46,721 [MainThread  ] [INFO ]  Processing giga.00432.txt.
2019-06-09 12:07:46,721 [MainThread  ] [INFO ]  Processing giga.00850.txt.
2019-06-09 12:07:46,722 [MainThread  ] [INFO ]  Processing giga.01349.txt.
2019-06-09 12:07:46,723 [MainThread  ] [INFO ]  Processing giga.00961.txt.
2019-06-09 12:07:46,724 [MainThread  ] [INFO ]  Processing giga.00473.txt.
2019-06-09 12:07:46,724 [

2019-06-09 12:07:46,800 [MainThread  ] [INFO ]  Processing giga.01874.txt.
2019-06-09 12:07:46,801 [MainThread  ] [INFO ]  Processing giga.00765.txt.
2019-06-09 12:07:46,801 [MainThread  ] [INFO ]  Processing giga.00198.txt.
2019-06-09 12:07:46,802 [MainThread  ] [INFO ]  Processing giga.01739.txt.
2019-06-09 12:07:46,803 [MainThread  ] [INFO ]  Processing giga.01801.txt.
2019-06-09 12:07:46,803 [MainThread  ] [INFO ]  Processing giga.01195.txt.
2019-06-09 12:07:46,804 [MainThread  ] [INFO ]  Processing giga.01914.txt.
2019-06-09 12:07:46,805 [MainThread  ] [INFO ]  Processing giga.01628.txt.
2019-06-09 12:07:46,805 [MainThread  ] [INFO ]  Processing giga.01163.txt.
2019-06-09 12:07:46,806 [MainThread  ] [INFO ]  Processing giga.00885.txt.
2019-06-09 12:07:46,807 [MainThread  ] [INFO ]  Processing giga.00959.txt.
2019-06-09 12:07:46,807 [MainThread  ] [INFO ]  Processing giga.00854.txt.
2019-06-09 12:07:46,808 [MainThread  ] [INFO ]  Processing giga.00550.txt.
2019-06-09 12:07:46,808 [

2019-06-09 12:07:46,884 [MainThread  ] [INFO ]  Processing giga.00964.txt.
2019-06-09 12:07:46,884 [MainThread  ] [INFO ]  Processing giga.01898.txt.
2019-06-09 12:07:46,885 [MainThread  ] [INFO ]  Processing giga.00794.txt.
2019-06-09 12:07:46,886 [MainThread  ] [INFO ]  Processing giga.00509.txt.
2019-06-09 12:07:46,886 [MainThread  ] [INFO ]  Processing giga.01397.txt.
2019-06-09 12:07:46,887 [MainThread  ] [INFO ]  Processing giga.00858.txt.
2019-06-09 12:07:46,888 [MainThread  ] [INFO ]  Processing giga.00527.txt.
2019-06-09 12:07:46,888 [MainThread  ] [INFO ]  Processing giga.01227.txt.
2019-06-09 12:07:46,889 [MainThread  ] [INFO ]  Processing giga.01029.txt.
2019-06-09 12:07:46,890 [MainThread  ] [INFO ]  Processing giga.01586.txt.
2019-06-09 12:07:46,890 [MainThread  ] [INFO ]  Processing giga.00224.txt.
2019-06-09 12:07:46,891 [MainThread  ] [INFO ]  Processing giga.01572.txt.
2019-06-09 12:07:46,892 [MainThread  ] [INFO ]  Processing giga.00641.txt.
2019-06-09 12:07:46,893 [

...

2019-06-09 12:07:48,652 [MainThread  ] [INFO ]  Processing giga.A.00484.txt.
2019-06-09 12:07:48,656 [MainThread  ] [INFO ]  Processing giga.A.00436.txt.
2019-06-09 12:07:48,657 [MainThread  ] [INFO ]  Processing giga.A.01182.txt.
2019-06-09 12:07:48,658 [MainThread  ] [INFO ]  Processing giga.A.00490.txt.
2019-06-09 12:07:48,659 [MainThread  ] [INFO ]  Processing giga.A.01153.txt.
2019-06-09 12:07:48,659 [MainThread  ] [INFO ]  Processing giga.A.01747.txt.
2019-06-09 12:07:48,660 [MainThread  ] [INFO ]  Processing giga.A.00903.txt.
2019-06-09 12:07:48,661 [MainThread  ] [INFO ]  Processing giga.A.01238.txt.
2019-06-09 12:07:48,662 [MainThread  ] [INFO ]  Processing giga.A.01058.txt.
2019-06-09 12:07:48,663 [MainThread  ] [INFO ]  Processing giga.A.01832.txt.
2019-06-09 12:07:48,664 [MainThread  ] [INFO ]  Processing giga.A.01383.txt.
2019-06-09 12:07:48,665 [MainThread  ] [INFO ]  Processing giga.A.00285.txt.
2019-06-09 12:07:48,666 [MainThread  ] [INFO ]  Processing giga.A.01528.txt.

2019-06-09 12:07:48,758 [MainThread  ] [INFO ]  Processing giga.A.00920.txt.
2019-06-09 12:07:48,759 [MainThread  ] [INFO ]  Processing giga.A.01587.txt.
2019-06-09 12:07:48,760 [MainThread  ] [INFO ]  Processing giga.A.00749.txt.
2019-06-09 12:07:48,761 [MainThread  ] [INFO ]  Processing giga.A.01839.txt.
2019-06-09 12:07:48,761 [MainThread  ] [INFO ]  Processing giga.A.00763.txt.
2019-06-09 12:07:48,763 [MainThread  ] [INFO ]  Processing giga.A.00558.txt.
2019-06-09 12:07:48,766 [MainThread  ] [INFO ]  Processing giga.A.00419.txt.
2019-06-09 12:07:48,767 [MainThread  ] [INFO ]  Processing giga.A.01533.txt.
2019-06-09 12:07:48,768 [MainThread  ] [INFO ]  Processing giga.A.00177.txt.
2019-06-09 12:07:48,769 [MainThread  ] [INFO ]  Processing giga.A.01170.txt.
2019-06-09 12:07:48,769 [MainThread  ] [INFO ]  Processing giga.A.00341.txt.
2019-06-09 12:07:48,770 [MainThread  ] [INFO ]  Processing giga.A.00731.txt.
2019-06-09 12:07:48,771 [MainThread  ] [INFO ]  Processing giga.A.00246.txt.

2019-06-09 12:07:48,855 [MainThread  ] [INFO ]  Processing giga.A.01179.txt.
2019-06-09 12:07:48,856 [MainThread  ] [INFO ]  Processing giga.A.01036.txt.
2019-06-09 12:07:48,856 [MainThread  ] [INFO ]  Processing giga.A.01061.txt.
2019-06-09 12:07:48,857 [MainThread  ] [INFO ]  Processing giga.A.01158.txt.
2019-06-09 12:07:48,859 [MainThread  ] [INFO ]  Processing giga.A.01003.txt.
2019-06-09 12:07:48,860 [MainThread  ] [INFO ]  Processing giga.A.00990.txt.
2019-06-09 12:07:48,860 [MainThread  ] [INFO ]  Processing giga.A.01681.txt.
2019-06-09 12:07:48,862 [MainThread  ] [INFO ]  Processing giga.A.01804.txt.
2019-06-09 12:07:48,863 [MainThread  ] [INFO ]  Processing giga.A.01054.txt.
2019-06-09 12:07:48,864 [MainThread  ] [INFO ]  Processing giga.A.01315.txt.
2019-06-09 12:07:48,865 [MainThread  ] [INFO ]  Processing giga.A.01555.txt.
2019-06-09 12:07:48,866 [MainThread  ] [INFO ]  Processing giga.A.00398.txt.
2019-06-09 12:07:48,867 [MainThread  ] [INFO ]  Processing giga.A.01130.txt.

2019-06-09 12:07:48,953 [MainThread  ] [INFO ]  Processing giga.A.01356.txt.
2019-06-09 12:07:48,954 [MainThread  ] [INFO ]  Processing giga.A.00480.txt.
2019-06-09 12:07:48,955 [MainThread  ] [INFO ]  Processing giga.A.00501.txt.
2019-06-09 12:07:48,956 [MainThread  ] [INFO ]  Processing giga.A.00044.txt.
2019-06-09 12:07:48,957 [MainThread  ] [INFO ]  Processing giga.A.01817.txt.
2019-06-09 12:07:48,958 [MainThread  ] [INFO ]  Processing giga.A.01317.txt.
2019-06-09 12:07:48,960 [MainThread  ] [INFO ]  Processing giga.A.01579.txt.
2019-06-09 12:07:48,960 [MainThread  ] [INFO ]  Processing giga.A.00134.txt.
2019-06-09 12:07:48,961 [MainThread  ] [INFO ]  Processing giga.A.00061.txt.
2019-06-09 12:07:48,962 [MainThread  ] [INFO ]  Processing giga.A.00395.txt.
2019-06-09 12:07:48,963 [MainThread  ] [INFO ]  Processing giga.A.01917.txt.
2019-06-09 12:07:48,964 [MainThread  ] [INFO ]  Processing giga.A.01690.txt.
2019-06-09 12:07:48,965 [MainThread  ] [INFO ]  Processing giga.A.00515.txt.

---------------------------------------------
1 ROUGE-1 Average_R: 0.30010 (95%-conf.int. 0.28945 - 0.31092)
1 ROUGE-1 Average_P: 0.31406 (95%-conf.int. 0.30333 - 0.32491)
1 ROUGE-1 Average_F: 0.29574 (95%-conf.int. 0.28538 - 0.30604)
---------------------------------------------
1 ROUGE-2 Average_R: 0.12691 (95%-conf.int. 0.11853 - 0.13570)
1 ROUGE-2 Average_P: 0.13285 (95%-conf.int. 0.12383 - 0.14202)
1 ROUGE-2 Average_F: 0.12487 (95%-conf.int. 0.11675 - 0.13326)
---------------------------------------------
1 ROUGE-3 Average_R: 0.06485 (95%-conf.int. 0.05809 - 0.07253)
1 ROUGE-3 Average_P: 0.06704 (95%-conf.int. 0.05972 - 0.07495)
1 ROUGE-3 Average_F: 0.06320 (95%-conf.int. 0.05644 - 0.07048)
---------------------------------------------
1 ROUGE-4 Average_R: 0.03703 (95%-conf.int. 0.03126 - 0.04348)
1 ROUGE-4 Average_P: 0.03873 (95%-conf.int. 0.03267 - 0.04566)
1 ROUGE-4 Average_F: 0.03634 (95%-conf.int. 0.03053 - 0.04278)
---------------------------------------------
1 ROUGE-L Aver